# **Proyecto 1: Sistema de Organización de Finanzas Personales**

## **Bibliotecas y Clases de Modelos**

In [92]:
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt

class User:
    def __init__(self, user_id, name, password):
        self.id = user_id
        self.name = name
        self.password = password

class Transaction:
    def __init__(self, trans_id, user_id, amount, category, date, is_deductible, is_income, description):
        self.id = trans_id
        self.user_id = user_id
        self.amount = amount
        self.category = category
        self.date = date
        self.is_deductible = is_deductible
        self.is_income = is_income
        self.description = description

class Data:
    def __init__(self, key, value):
        self.key = key
        self.value = value


## **Árbol B**

In [93]:
class BTreeNode:
    def __init__(self, t, leaf):
        self.t = t
        self.keys = [None] * (2 * t - 1)
        self.C = [None] * (2 * t)
        self.n = 0
        self.leaf = leaf

    def insertNonFull(self, data):
        i = self.n - 1
        if self.leaf:
            while i >= 0 and self.keys[i].key > data.key:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = data
            self.n += 1
        else:
            while i >= 0 and self.keys[i].key > data.key:
                i -= 1
            if self.C[i + 1].n == 2 * self.t - 1:
                self.splitChild(i + 1, self.C[i + 1])
                if self.keys[i + 1].key < data.key:
                    i += 1
            self.C[i + 1].insertNonFull(data)

    def splitChild(self, i, y):
        z = BTreeNode(y.t, y.leaf)
        z.n = self.t - 1
        for j in range(self.t - 1):
            z.keys[j] = y.keys[j + self.t]
        if not y.leaf:
            for j in range(self.t):
                z.C[j] = y.C[j + self.t]
        y.n = self.t - 1
        for j in range(self.n, i, -1):
            self.C[j + 1] = self.C[j]
        self.C[i + 1] = z
        for j in range(self.n - 1, i - 1, -1):
            self.keys[j + 1] = self.keys[j]
        self.keys[i] = y.keys[self.t - 1]
        self.n += 1

    def traverse(self, output_list):
        for i in range(self.n):
            if not self.leaf:
                self.C[i].traverse(output_list)
            output_list.append(self.keys[i])
        if not self.leaf:
            self.C[self.n].traverse(output_list)

    def search(self, k):
        i = 0
        while i < self.n and k > self.keys[i].key:
            i += 1
        if i < self.n and k == self.keys[i].key:
            return self.keys[i]
        if self.leaf:
            return None
        return self.C[i].search(k)

class BTree:
    def __init__(self, t):
        self.root = None
        self.t = t

    def traverse(self):
        output_list = []
        if self.root != None:
            self.root.traverse(output_list)
        return output_list

    def search(self, k):
        return None if self.root == None else self.root.search(k)

    def insert(self, data):
        if self.root == None:
            self.root = BTreeNode(self.t, True)
            self.root.keys[0] = data
            self.root.n = 1
        else:
            if self.root.n == 2 * self.t - 1:
                s = BTreeNode(self.t, False)
                s.C[0] = self.root
                s.splitChild(0, self.root)
                i = 0
                if s.keys[0].key < data.key:
                    i += 1
                s.C[i].insertNonFull(data)
                self.root = s
            else:
                self.root.insertNonFull(data)


## **Merge Sort**

In [94]:
def merge_sort(arr):
    """
    Implementación de Merge Sort (ordenamiento por mezcla)
    
    Args:
        arr: Lista a ordenar
    
    Returns:
        Lista ordenada
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]
    
    left_sorted = merge_sort(left_half)
    right_sorted = merge_sort(right_half)
    
    return merge(left_sorted, right_sorted)

def merge(left, right):
    """
    Mezcla dos listas ordenadas en una sola lista ordenada
    """
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
            
    while i < len(left):
        result.append(left[i])
        i += 1
        
    while j < len(right):
        result.append(right[j])
        j += 1
    
    return result

## **Búsqueda Binaria**

In [95]:
def busqueda_binaria(lista, objetivo):
    izquierda = 0
    derecha = len(lista) - 1

    while izquierda <= derecha:
        medio = (izquierda + derecha) // 2
        valor_medio = lista[medio]

        if valor_medio == objetivo:
            return medio
        elif valor_medio < objetivo:
            izquierda = medio + 1
        else:
            derecha = medio - 1

    return -1

## **Gestor de Archivos**

In [96]:
class FileController:
    def __init__(self, users_file="users.json", transactions_file="transactions.json"):
        self.users_file = users_file
        self.transactions_file = transactions_file
        self.transaction_tree = BTree(3)  # degree 3 (can adjust)
        self._initialize_files()
        self._load_transactions_to_btree()

    def _initialize_files(self):
        for file in [self.users_file, self.transactions_file]:
            if not os.path.exists(file):
                with open(file, "w") as f:
                    json.dump([], f)

    def load_json(self, file):
        with open(file, "r") as f:
            return json.load(f)

    def save_json(self, file, data):
        with open(file, "w") as f:
            json.dump(data, f, indent=4)

    def verify_user(self, user_id, password):
        users = self.load_json(self.users_file)
        for u in users:
            if u["id"] == user_id and u["password"] == password:
                return User(u["id"], u["name"], u["password"])
        return None

    def get_next_id(self):
        transactions = self.load_json(self.transactions_file)
        if not transactions:
            return 1
        return max(t["id"] for t in transactions) + 1

    # ------------------- Transaction Handling -------------------
    def save_transaction(self, transaction):
        transactions = self.load_json(self.transactions_file)
        transactions.append(transaction.__dict__)
        self.save_json(self.transactions_file, transactions)
        self.transaction_tree.insert(Data(transaction.id, transaction))

    def _load_transactions_to_btree(self):
        """Loads existing transactions from file into the B-Tree."""
        transactions = self.load_json(self.transactions_file)
        for t in transactions:
            obj = Transaction(
                t["id"], t["user_id"], t["amount"], t["category"],
                t["date"], t["is_deductible"], t["is_income"], t["description"]
            )
            self.transaction_tree.insert(Data(obj.id, obj))

    def find_transaction(self, trans_id):
        node = self.transaction_tree.search(trans_id)
        return node.value if node else None

    def update_transaction(self, trans_id, field, new_value):
        transactions = self.load_json(self.transactions_file)
        updated = False
        for t in transactions:
            if t["id"] == trans_id:
                t[field] = new_value
                updated = True
                break
        if updated:
            self.save_json(self.transactions_file, transactions)
            # Also update in B-Tree
            trans = self.find_transaction(trans_id)
            if trans:
                setattr(trans, field, new_value)
        return updated

    def delete_transaction(self, trans_id):
        """Deletes from file (B-Tree deletion is complex, omitted for simplicity)."""
        transactions = self.load_json(self.transactions_file)
        new_data = [t for t in transactions if t["id"] != trans_id]
        if len(new_data) == len(transactions):
            return False
        self.save_json(self.transactions_file, new_data)
        # Rebuild tree for simplicity (no full delete implemented)
        self.transaction_tree = BTree(3)
        self._load_transactions_to_btree()
        return True

    # ------------------- Merge Sort -------------------
    def merge_sort(self, items, key="amount"):
        if len(items) <= 1:
            return items
        mid = len(items) // 2
        left = self.merge_sort(items[:mid], key)
        right = self.merge_sort(items[mid:], key)
        return self._merge(left, right, key)

    def _merge(self, left, right, key):
        result = []
        i = j = 0
        while i < len(left) and j < len(right):
            if getattr(left[i], key) <= getattr(right[j], key):
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        result += left[i:]
        result += right[j:]
        return result

## **Sistema**

In [ ]:
class System:
    def __init__(self):
        self.files = FileController()
        self.current_user = None

    # ------------------- LOGIN -------------------
    def login(self, user_id, password):
        user = self.files.verify_user(user_id, password)
        if user:
            self.current_user = user
            print(f"✅ Welcome {user.name}")
            return True
        else:
            print("❌ Invalid user ID or password.")
            return False

    def logout(self):
        if self.current_user:
            print(f"👋 Logged out: {self.current_user.name}")
            self.current_user = None
        else:
            print("⚠️ No active session.")

    # ------------------- MENU -------------------
    def show_menu(self):
        print("\n===== SYSTEM MENU =====")
        print("1. Register New Transaction")
        print("2. View Transaction Data")
        print("3. Modify Transaction Data")
        print("4. Delete Transaction")
        print("5. Generate Transaction Report")
        print("6. Calculate Taxes (Pending)")
        print("7. Logout")
        try:
            option = int(input("Select an option: "))
        except ValueError:
            print("⚠️ Invalid input. Please enter a number.")
            option = 0
        return option

    # ------------------- REGISTER TRANSACTION -------------------
    def register_transaction(self):
        if not self.current_user:
            print("⚠️ You must log in first.")
            return

        amount = float(input("Amount: "))
        is_deductible = int(input("Is deductible? (0=No, 1=Yes): "))
        is_income = int(input("Is income? (0=Expense, 1=Income): "))

        categories = ["Salary", "Donation", "Investment", "Housing", "Food",
                      "Health", "Transport", "Education", "Debts", "Other"]
        for i, cat in enumerate(categories, 1):
            print(f"{i}. {cat}")
        category = categories[int(input("Select category: ")) - 1]
        description = input("Description: ")

        trans_id = self.files.get_next_id()
        date = datetime.now().strftime("%Y-%m-%d")

        new_trans = Transaction(
            trans_id, self.current_user.id, amount, category, date,
            is_deductible, is_income, description
        )

        self.files.save_transaction(new_trans)
        print(f"✅ Transaction #{trans_id} registered successfully.")

    # ------------------- VIEW TRANSACTION -------------------
    def view_transaction(self):
        trans_id = int(input("Enter transaction ID: "))
        trans = self.files.find_transaction(trans_id)
        if not trans or trans.user_id != self.current_user.id:
            print("❌ Transaction not found or not owned by this user.")
            return

        print(f"\n--- Transaction #{trans.id} ---")
        print(f"Category: {trans.category}")
        print(f"Amount: ${trans.amount}")
        print(f"Date: {trans.date}")
        print(f"Deductible: {'Yes' if trans.is_deductible else 'No'}")
        print(f"Type: {'Income' if trans.is_income else 'Expense'}")
        print(f"Description: {trans.description}")

    # ------------------- MODIFY TRANSACTION -------------------
    def modify_transaction(self):
        trans_id = int(input("Enter transaction ID: "))
        trans = self.files.find_transaction(trans_id)
        if not trans or trans.user_id != self.current_user.id:
            print("❌ Transaction not found or not owned by this user.")
            return

        while True:
            print("\nSelect field to modify:")
            print("1. Amount")
            print("2. Is Deductible")
            print("3. Is Income")
            print("4. Category")
            print("5. Description")
            print("6. Exit")
            op = int(input("Option: "))

            if op == 6:
                break

            field_map = {
                1: "amount",
                2: "is_deductible",
                3: "is_income",
                4: "category",
                5: "description"
            }

            field = field_map.get(op)
            if not field:
                print("Invalid option.")
                continue

            new_value = input(f"Enter new value for {field}: ")
            if field in ["amount"]:
                new_value = float(new_value)
            elif field in ["is_deductible", "is_income"]:
                new_value = int(new_value)
            elif field == "category":
                categories = ["Salary", "Donation", "Investment", "Housing", "Food",
                              "Health", "Transport", "Education", "Debts", "Other"]
                for i, cat in enumerate(categories, 1):
                    print(f"{i}. {cat}")
                new_value = categories[int(new_value) - 1]

            self.files.update_transaction(trans_id, field, new_value)
            print("✅ Field updated successfully.")

    # ------------------- DELETE TRANSACTION -------------------
    def delete_transaction(self):
        trans_id = int(input("Enter transaction ID to delete: "))
        trans = self.files.find_transaction(trans_id)
        if not trans or trans.user_id != self.current_user.id:
            print("❌ Transaction not found or not owned by this user.")
            return

        if self.files.delete_transaction(trans_id):
            print(f"🗑️ Transaction #{trans_id} deleted successfully.")
        else:
            print("❌ Transaction not found.")

    # ------------------- REPORT GENERATION -------------------
    def generate_report(self):
        print("\nGenerate report by:")
        print("1. Amount")
        print("2. Category")
        print("3. Date")
        print("4. Is Deductible")
        print("5. Is Income")

        op = int(input("Select option: "))
        # Only transactions from current user
        all_trans = [node.value for node in self.files.transaction_tree.traverse()
                     if node.value.user_id == self.current_user.id]
        results = []

        if op == 1:
            min_a = float(input("Minimum amount: "))
            max_a = float(input("Maximum amount: "))
            results = [t for t in all_trans if min_a <= t.amount <= max_a]
            title = f"Report: Transactions between ${min_a} and ${max_a}"
        elif op == 2:
            cat = input("Enter category: ")
            results = [t for t in all_trans if t.category.lower() == cat.lower()]
            title = f"Report: Transactions in category '{cat}'"
        elif op == 3:
            start = input("From (YYYY-MM-DD): ")
            end = input("To (YYYY-MM-DD): ")
            results = [t for t in all_trans if start <= t.date <= end]
            title = f"Report: Transactions from {start} to {end}"
        elif op == 4:
            results = [t for t in all_trans if t.is_deductible == 1]
            title = "Report: Deductible Transactions"
        elif op == 5:
            results = [t for t in all_trans if t.is_income == 1]
            title = "Report: Income Transactions"
        else:
            print("Invalid option.")
            return

        results = self.files.merge_sort(results, "amount")

        filename = f"transactions_report_user_{self.current_user.id}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"{title}\n")
            f.write(f"User: {self.current_user.name}\n\n")
            for t in results:
                f.write(f"ID: {t.id}\n")
                f.write(f"Category: {t.category}\n")
                f.write(f"Amount: ${t.amount}\n")
                f.write(f"Date: {t.date}\n")
                f.write(f"Deductible: {'Yes' if t.is_deductible else 'No'}\n")
                f.write(f"Type: {'Income' if t.is_income else 'Expense'}\n")
                f.write(f"Description: {t.description}\n")
                f.write("-" * 40 + "\n")
        print(f"📄 Report generated: {filename}")


## **Ejecución**

In [ ]:
system = System()
si = system.login(1, "1234")

while True and si is True:
    option = system.show_menu()
    if option == 1:
        system.register_transaction()
    elif option == 2:
        system.view_transaction()
    elif option == 3:
        system.modify_transaction()
    elif option == 4:
        system.delete_transaction()
    elif option == 5:
        system.generate_report()
    elif option == 6:
        print("Tax calculation pending implementation.")
    elif option == 7:
        system.logout()
        break
    else:
        print("Invalid option.")

# CONSTRUIR LOS ÁRBOLES SOLO 


✅ Welcome Abraham Gonzalez

===== SYSTEM MENU =====
1. Register New Transaction
2. View Transaction Data
3. Modify Transaction Data
4. Delete Transaction
5. Generate Transaction Report
6. Calculate Taxes (Pending)
7. Logout
1. Salary
2. Donation
3. Investment
4. Housing
5. Food
6. Health
7. Transport
8. Education
9. Debts
10. Other
✅ Transaction #1002 registered successfully.

===== SYSTEM MENU =====
1. Register New Transaction
2. View Transaction Data
3. Modify Transaction Data
4. Delete Transaction
5. Generate Transaction Report
6. Calculate Taxes (Pending)
7. Logout

--- Transaction #1 ---
Category: Investment
Amount: $3811.8
Date: 2024-07-20
Deductible: No
Type: Expense
Description: Retiro de efectivo

===== SYSTEM MENU =====
1. Register New Transaction
2. View Transaction Data
3. Modify Transaction Data
4. Delete Transaction
5. Generate Transaction Report
6. Calculate Taxes (Pending)
7. Logout
👋 Logged out: Abraham Gonzalez
